# 03 — Entrenamiento y Evaluación (SUPERVISED y CONTINUAL con EWC/NAIVE)

Este notebook entrena un modelo **SNN** para **regresión del ángulo de dirección (steering)** en dos protocolos:

- **Supervised** sobre `circuito1`.
- **Continual** con dos tareas secuenciales `circuito1 → circuito2` usando:
  - **EWC** (consolidación elástica de pesos), o
  - **NAIVE** (baseline sin penalización; equivalente a λ=0).

> **Requisitos previos**: Ejecuta `01_DATA_QC_PREP.ipynb` para generar `train/val/test.csv` y `tasks.json`.


In [1]:
# =============================================================================
# Imports y setup
# =============================================================================
from pathlib import Path
import sys, json, torch

# Detecta la raíz del repo (si estás dentro de notebooks/, sube un nivel)
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Utilidades y módulos del proyecto
from src.utils import set_seeds, load_preset, make_loaders_from_csvs, ImageTransform
from src.models import SNNVisionRegressor
from src.training import TrainConfig, train_supervised, _permute_if_needed
from src.methods.ewc import EWC, EWCConfig

# Dispositivo (CUDA si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT, device

SEED = 42

In [2]:
# Transformación de imagen
# IMPORTANTE: usa argumentos **posicionales** (w, h, to_gray, crop_top)
# Evita keywords tipo target_w/target_h porque la clase no los define.
tfm = ImageTransform(160, 80, True, None)

In [3]:
# =============================================================================
# Verificación de datos
# =============================================================================
RAW = ROOT/"data"/"raw"/"udacity"
PROC = ROOT/"data"/"processed"

# Comprueba que existen los CSV por split de cada circuito
for run in ["circuito1","circuito2"]:
    for part in ["train","val","test"]:
        path = PROC/run/f"{part}.csv"
        if not path.exists():
            raise FileNotFoundError(f"Falta {path}. Ejecuta 01_DATA_QC_PREP.ipynb primero.")
print("OK splits encontrados")

OK splits encontrados


In [4]:
# =============================================================================
# Función para crear loaders de una tarea dada (respeta cfg del preset)
# =============================================================================
def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed,
                   num_workers=12, prefetch_factor=8,  # ↑ antes 8 y 4
                   pin_memory=True, persistent_workers=True):
    from pathlib import Path
    name  = task["name"]
    paths = task["paths"]
    return make_loaders_from_csvs(
        base_dir=RAW/name,
        train_csv=Path(paths["train"]),
        val_csv=Path(paths["val"]),
        test_csv=Path(paths["test"]),
        batch_size=batch_size,
        encoder=encoder,
        T=T,
        gain=gain,
        tfm=tfm,
        seed=seed,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=persistent_workers,
        prefetch_factor=prefetch_factor,
    )


In [5]:
from src.training import _permute_if_needed  # importa el helper del modelo

# =============================================================================
# Helper de evaluación (permuta a (T,B,C,H,W) y usa copias no bloqueantes)
# =============================================================================
def eval_loader(loader, model, device):
    """Calcula MAE/MSE promediados sobre todo el loader.

    - El DataLoader produce (B, T, C, H, W)
    - El modelo espera      (T, B, C, H, W)
    """
    model.eval()  # modo evaluación: desactiva dropout/batchnorm, etc.
    mae_sum = 0.0
    mse_sum = 0.0
    n = 0

    # Un solo no_grad() fuera del bucle para minimizar overhead
    with torch.no_grad():
        for x, y in loader:
            # (B,T,C,H,W) -> (T,B,C,H,W), y luego a GPU con non_blocking=True
            x = _permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            y_hat = model(x)

            # Acumula MAE/MSE ponderados por el tamaño real del batch
            mae_sum += torch.mean(torch.abs(y_hat - y)).item() * len(y)
            mse_sum += torch.mean((y_hat - y) ** 2).item() * len(y)
            n += len(y)

    return (mae_sum / max(n, 1)), (mse_sum / max(n, 1))


In [6]:
# =============================================================================
# Cargar orden de tareas (continual) desde tasks.json
# =============================================================================
with open(PROC/"tasks.json","r",encoding="utf-8") as f:
    tasks_json = json.load(f)

# task_list = [{'name': 'circuito1', 'paths': {...}}, {'name': 'circuito2', 'paths': {...}}]
task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
task_list

[{'name': 'circuito1',
  'paths': {'train': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito1/train.csv',
   'val': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito1/val.csv',
   'test': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito1/test.csv'}},
 {'name': 'circuito2',
  'paths': {'train': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito2/train.csv',
   'val': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito2/val.csv',
   'test': '/home/cesar/proyectos/TFM_SNN/data/processed/circuito2/test.csv'}}]

In [7]:
# ====================== run_continual que carga el preset internamente ======================
from pathlib import Path
import json, torch
from src.utils import load_preset, make_loaders_from_csvs, set_seeds
from src.training import TrainConfig, train_supervised
from src.models import SNNVisionRegressor
from src.methods.ewc import EWC, EWCConfig

def run_continual(
    preset: str,                 # "fast" | "std" | "accurate"
    method: str,                 # "ewc" | "naive"
    lam: float | None,           # λ si EWC; None si naive
    seed: int,
    encoder: str,                # "rate" | "latency"
    tfm,                         # ImageTransform a usar
    fisher_batches_by_preset: dict[str,int] | None = None,
):
    # Carga del preset específico para ESTE run
    cfg = load_preset(ROOT/"configs"/"presets.yaml", preset)
    T     = int(cfg["T"])
    gain  = float(cfg["gain"])
    lr    = float(cfg["lr"])
    epochs= int(cfg["epochs"])
    bs    = int(cfg["batch_size"])
    use_amp = bool(cfg["amp"])

    # Fisher batches según preset (si no se da, usa 100 por defecto)
    fb = 100
    if fisher_batches_by_preset and preset in fisher_batches_by_preset:
        fb = int(fisher_batches_by_preset[preset])

    set_seeds(seed)

    # Modelo y método
    model = SNNVisionRegressor(in_channels=1, lif_beta=0.95)
    ewc = None
    if method == "ewc":
        assert lam is not None, "Para EWC debes pasar λ (lam)"
        ewc = EWC(model, EWCConfig(lambd=float(lam), fisher_batches=fb))

    # Tareas (asumimos task_list ya existe en el notebook)
    out_tag = f"continual_{preset}_{method}" + (f"_lam_{lam:.0e}" if method=="ewc" else "") + f"_{encoder}_seed_{seed}"
    out_dir = ROOT/"outputs"/out_tag
    out_dir.mkdir(parents=True, exist_ok=True)

    # entrenamiento secuencial
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_fn = torch.nn.MSELoss()
    tcfg = TrainConfig(epochs=epochs, batch_size=bs, lr=lr, amp=use_amp, seed=seed)

    results = {}
    seen = []
    for i, t in enumerate(task_list):
        name = t["name"]; paths = t["paths"]
        tr, va, te = make_loaders_from_csvs(
            base_dir=RAW/name,
            train_csv=Path(paths["train"]),
            val_csv=Path(paths["val"]),
            test_csv=Path(paths["test"]),
            batch_size=bs,
            encoder=encoder,
            T=T,
            gain=gain,
            tfm=tfm,
            seed=seed,
            num_workers=12, prefetch_factor=8, pin_memory=True, persistent_workers=True,
        )

        _ = train_supervised(
            model, tr, va, loss_fn, tcfg,
            out_dir/f"task_{i+1}_{name}",
            method=ewc if method=="ewc" else None
        )

        if method=="ewc":
            print("Estimando Fisher…")
            ewc.estimate_fisher(va, loss_fn, device=device)

        # evaluación tarea actual
        te_mae, te_mse = eval_loader(te, model, device)
        results[name] = {"test_mae": te_mae, "test_mse": te_mse}
        seen.append((name, te))

        # reevaluación tareas previas (olvido)
        for pname, p_loader in seen[:-1]:
            p_mae, p_mse = eval_loader(p_loader, model, device)
            results[pname][f"after_{name}_mae"] = p_mae
            results[pname][f"after_{name}_mse"] = p_mse

    (out_dir/"continual_results.json").write_text(json.dumps(results, indent=2), encoding="utf-8")
    return out_dir, results


In [8]:
# ===================== Demo opcional (1 corrida) =====================
RUN_DEMO = False  # pon True para probar 1 run rápido

if RUN_DEMO:
    preset_demo  = "fast"
    method_demo  = "ewc"     # "ewc" | "naive"
    lam_demo     = 1e9       # si method_demo=="ewc"
    seed_demo    = 42

    demo_cfg = load_preset(ROOT / "configs" / "presets.yaml", preset_demo)
    encoder_demo = demo_cfg["encoder"]   # "rate" o "latency"

    out_path, res = run_continual(
        preset=preset_demo,
        method=method_demo,
        lam=(lam_demo if method_demo == "ewc" else None),
        seed=seed_demo,
        encoder=encoder_demo,
        tfm=tfm,
        fisher_batches_by_preset={"fast": 200, "std": 200},
    )
    print("OK:", out_path)
    res


In [ ]:
# ====================== CONFIG POR DEFECTO PARA LAS COMPARATIVAS ======================
# Recomendaciones acordadas:
# - fast: EWC λ=1e9 (estable). Extra: λ=1e8 (mejor T2, algo más de olvido)
# - std : EWC λ=1e7 (baseline estable). Extra: λ=3e7 (mejor T2, más olvido)

EWC_DEFAULTS = {
    "fast": {"primary": [3e8, 1e9], "extra": [1e10]},  # 1e10 como extra
    "std":  {"primary": [1e9],       "extra": [3e9]},  # 1e9 principal
}

# EWC_DEFAULTS = {
#     "fast": {"primary": [3e8, 1e9, 1e10], "extra": []},
#     "std":  {"primary": [3e8, 1e9, 3e9, 1e10], "extra": []},
# }

# EWC_DEFAULTS = {
#     "fast": {"primary": [3e8, 1e9, 1e10], "extra": []},
#     "std":  {"primary": [1e7, 3e7, 1e8],  "extra": []},
# }
# INCLUDE_NAIVE    = True     # añadimos baseline sin EWC
# INCLUDE_EXTRAS   = True    # desactivado porque ya listamos primarios exactos

# EWC_DEFAULTS = {
#     "fast":     {"primary": [1e9], "extra": [1e8]},
#     "std":      {"primary": [1e7], "extra": [3e7]},
    # Añadir "accurate":
    # "accurate": {"primary": [1e7], "extra": []},
# }

INCLUDE_NAIVE    = True          # añade baseline sin EWC
INCLUDE_EXTRAS   = True          # activa los λ "extra" por preset
SEEDS            = [42, 43, 44]  # multisemillas para medias/σ
ENCODERS         = ["rate"]      # luego podrás añadir "latency"
# FISHER_BY_PRESET = {"fast": 200, "std": 600}  # estabiliza el cálculo de Fisher
FISHER_BY_PRESET = {"fast": 400, "std": 600}  # estabiliza el cálculo de Fisher

# Elige qué presets lanzar
PRESETS_TO_RUN = ["fast", "std"]  # añade "accurate" si lo necesitas más adelante

# ---- Construcción del plan de ejecuciones ----
runs_plan = []
for preset_i in PRESETS_TO_RUN:
    # EWC primary
    for lam in EWC_DEFAULTS[preset_i]["primary"]:
        runs_plan.append((preset_i, "ewc", lam))
    # EWC extras (opcionales)
    if INCLUDE_EXTRAS:
        for lam in EWC_DEFAULTS[preset_i]["extra"]:
            runs_plan.append((preset_i, "ewc", lam))
    # Baseline sin EWC
    if INCLUDE_NAIVE:
        runs_plan.append((preset_i, "naive", None))

print("Plan de runs (preset, método, λ):")
for preset_i, method_i, lam_i in runs_plan:
    print(f"  {preset_i:>7}  {method_i:>5}  λ={lam_i}")
print("Semillas:", SEEDS, " | Encoders:", ENCODERS)
print("Fisher batches por preset:", FISHER_BY_PRESET)


Plan de runs (preset, método, λ):
     fast    ewc  λ=300000000.0
     fast    ewc  λ=1000000000.0
     fast    ewc  λ=10000000000.0
     fast  naive  λ=None
      std    ewc  λ=300000000.0
      std    ewc  λ=1000000000.0
      std    ewc  λ=3000000000.0
      std    ewc  λ=10000000000.0
      std  naive  λ=None
Semillas: [42, 43, 44]  | Encoders: ['rate']
Fisher batches por preset: {'fast': 200, 'std': 1000}


In [10]:
# ====================== DRIVER MULTISEMILLAS (usa la CONFIG de arriba) ======================
for enc in ENCODERS:
    for seed in SEEDS:
        for preset_i, method_i, lam_i in runs_plan:
            print(f"\n=== RUN: preset={preset_i} | method={method_i} | λ={lam_i} | seed={seed} | encoder={enc} ===")
            out_path, _ = run_continual(
                preset=preset_i,
                method=method_i,
                lam=(lam_i if method_i == "ewc" else None),
                seed=seed,
                encoder=enc,
                tfm=tfm,  # definido en tu celda de setup
                fisher_batches_by_preset=FISHER_BY_PRESET,
            )
            print("OK:", out_path)
print("\n✅ Listo. Ejecuta las celdas de resumen.")



=== RUN: preset=fast | method=ewc | λ=300000000.0 | seed=42 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_3e+08_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=1000000000.0 | seed=42 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_1e+09_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=10000000000.0 | seed=42 | encoder=rate ===


...

Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_ewc_lam_3e+09_rate_seed_44

=== RUN: preset=std | method=ewc | λ=10000000000.0 | seed=44 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_ewc_lam_1e+10_rate_seed_44

=== RUN: preset=std | method=naive | λ=None | seed=44 | encoder=rate ===


OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_naive_rate_seed_44

✅ Listo. Ejecuta las celdas de resumen.


In [11]:
# =============================================================================
# Resumen comparativo de todos los continual_* en outputs/
# =============================================================================
import re, json
from pathlib import Path
import pandas as pd

def parse_exp_name(name: str):
    """
    Extrae preset, método, lambda, encoder y seed del nombre de carpeta:

      continual_<preset>_<method>[_lam_<lambda>]_<_encoder>[_seed_<seed>]

    Ejemplos:
      continual_fast_naive_rate_seed_42
      continual_fast_ewc_lam_1e+08_rate_seed_42
      continual_std_ewc_lam_3e+07_latency_seed_43
    """
    m = re.match(
        r"continual_(?P<preset>\w+)_(?P<method>ewc|naive)"
        r"(?:_lam_(?P<lambda>[^_]+))?_(?P<enc>[^_]+)"
        r"(?:_seed_(?P<seed>\d+))?$",
        name
    )
    meta = {"preset": None, "method": None, "lambda": None, "encoder": None, "seed": None}
    if m:
        d = m.groupdict()
        meta.update({
            "preset": d["preset"],
            "method": d["method"],
            "lambda": d.get("lambda"),
            "encoder": d.get("enc"),
            "seed": d.get("seed"),
        })
    return meta

rows = []
root_out = ROOT / "outputs"

for exp_dir in sorted(root_out.glob("continual_*")):
    name = exp_dir.name
    meta = parse_exp_name(name)

    # Saltar nombres no reconocidos (runs muy antiguos)
    if meta["preset"] is None:
        continue

    results_path = exp_dir / "continual_results.json"
    if not results_path.exists():
        continue

    with open(results_path, "r", encoding="utf-8") as f:
        res = json.load(f)

    # Detectar tareas: la "última" es la que NO tiene claves 'after_*'
    task_names = list(res.keys())
    if len(task_names) < 2:
        continue

    def is_last(d):  # no tiene after_*
        return not any(k.startswith("after_") for k in d.keys())

    last_task = None
    first_task = None
    for tn in task_names:
        if is_last(res[tn]):
            last_task = tn
        else:
            first_task = tn

    # Fallback por si no se identifica bien
    if first_task is None or last_task is None:
        task_names_sorted = sorted(task_names)
        first_task = task_names_sorted[0]
        last_task  = task_names_sorted[-1]

    c1, c2 = first_task, last_task

    c1_test_mae = float(res[c1].get("test_mae", float("nan")))
    c2_test_mae = float(res[c2].get("test_mae", float("nan")))
    after_key_mae = f"after_{c2}_mae"
    c1_after_c2_mae = float(res[c1].get(after_key_mae, float("nan")))

    forgetting_abs = c1_after_c2_mae - c1_test_mae
    forgetting_rel = (forgetting_abs / c1_test_mae * 100.0) if c1_test_mae == c1_test_mae else float("nan")

    rows.append({
        "exp": name,
        "preset": meta["preset"],
        "method": meta["method"],
        "lambda": meta["lambda"] if meta["method"] == "ewc" else None,
        "encoder": meta["encoder"],
        "seed": int(meta["seed"]) if meta["seed"] is not None else None,
        "c1_name": c1,
        "c2_name": c2,
        "c1_mae": c1_test_mae,
        "c1_after_c2_mae": c1_after_c2_mae,
        "c1_forgetting_mae_abs": forgetting_abs,
        "c1_forgetting_mae_rel_%": forgetting_rel,
        "c2_mae": c2_test_mae,
    })

df = pd.DataFrame(rows)

# Asegura columnas numéricas auxiliares
if "lambda_num" not in df.columns:
    df["lambda_num"] = pd.to_numeric(df["lambda"], errors="coerce")  # '1e+08' -> 1e+08 ; NAIVE -> NaN

# Deja 'seed' como entero y elimina 'seed_num' si existe
df["seed"] = pd.to_numeric(df["seed"], errors="coerce").astype("Int64")
if "seed_num" in df.columns:
    df = df.drop(columns=["seed_num"])

# Ordenar: preset, method, encoder, lambda_num (NaN al final), seed
df = df.sort_values(
    by=["preset", "method", "encoder", "lambda_num", "seed"],
    na_position="last",
    ignore_index=True,
)

df


,exp,preset,method,lambda,encoder,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_fast_ewc_lam_1e+07_rate_seed_42,fast,ewc,1e+07,rate,42,circuito1,circuito2,0.083898,0.083882,-0.000016,-0.018746,0.179655,1.000000e+07
1,continual_fast_ewc_lam_1e+07_rate_seed_43,fast,ewc,1e+07,rate,43,circuito1,circuito2,0.088412,0.088367,-0.000046,-0.051535,0.182569,1.000000e+07
2,continual_fast_ewc_lam_1e+07_rate_seed_44,fast,ewc,1e+07,rate,44,circuito1,circuito2,0.080361,0.190667,0.110306,137.262854,0.177495,1.000000e+07
3,continual_fast_ewc_lam_3e+07_rate_seed_42,fast,ewc,3e+07,rate,42,circuito1,circuito2,0.083898,0.083890,-0.000009,-0.010255,0.179659,3.000000e+07
4,continual_fast_ewc_lam_3e+07_rate_seed_43,fast,ewc,3e+07,rate,43,circuito1,circuito2,0.088412,0.088421,0.000009,0.009852,0.182635,3.000000e+07
5,continual_fast_ewc_lam_3e+07_rate_seed_44,fast,ewc,3e+07,rate,44,circuito1,circuito2,0.080361,0.089880,0.009519,11.845444,0.171028,3.000000e+07
6,continual_fast_ewc_lam_1e+08_rate_seed_42,fast,ewc,1e+08,rate,42,circuito1,circuito2,0.084650,0.104289,0.019639,23.200502,0.179869,1.000000e+08
7,continual_fast_ewc_lam_1e+08_rate_seed_43,fast,ewc,1e+08,rate,43,circuito1,circuito2,0.087956,0.087769,-0.000187,-0.212480,0.181760,1.000000e+08
8,continual_fast_ewc_lam_1e+08_rate_seed_44,fast,ewc,1e+08,rate,44,circuito1,circuito2,0.086885,0.088402,0.001517,1.746049,0.175735,1.000000e+08
9,continual_fast_ewc_lam_3e+08_rate_seed_42,fast,ewc,3e+08,rate,42,circuito1,circuito2,0.077664,0.100547,0.022883,29.463485,0.197777,3.000000e+08


In [12]:
# ====================== Vista agregada (media±std por preset/method/λ/encoder) ======================
import pandas as pd

# Métricas a agregar
cols_metrics = ["c1_mae", "c1_after_c2_mae", "c1_forgetting_mae_abs", "c1_forgetting_mae_rel_%", "c2_mae"]

# Copia y asegura columna numérica auxiliar para ordenar por λ
gdf = df.copy()
if "lambda_num" not in gdf.columns:
    gdf["lambda_num"] = pd.to_numeric(gdf["lambda"], errors="coerce")  # NA para NAIVE

# Agregación: media, std y número de corridas (semillas) por combinación
agg = (
    gdf
    .groupby(["preset", "method", "encoder", "lambda", "lambda_num"], dropna=False)[cols_metrics]
    .agg(["mean", "std", "count"])
    .reset_index()
)

# Aplanar nombres de columnas (de MultiIndex a una sola capa)
agg.columns = [
    "_".join(filter(None, map(str, col))).rstrip("_")
    for col in agg.columns.to_flat_index()
]

# Ordena por preset/method/encoder/λ_num (NaN al final ⇒ NAIVE al final de su grupo)
agg = agg.sort_values(
    by=["preset", "method", "encoder", "lambda_num"],
    na_position="last",
    ignore_index=True,
)

# (Opcional) guardar a CSV
summary_dir = ROOT / "outputs" / "summary"
summary_dir.mkdir(parents=True, exist_ok=True)
agg.to_csv(summary_dir / "continual_summary_agg.csv", index=False)
print("Guardado:", summary_dir / "continual_summary_agg.csv")

agg


Guardado: /home/cesar/proyectos/TFM_SNN/outputs/summary/continual_summary_agg.csv


,preset,method,encoder,lambda,lambda_num,c1_mae_mean,c1_mae_std,c1_mae_count,c1_after_c2_mae_mean,c1_after_c2_mae_std,c1_after_c2_mae_count,c1_forgetting_mae_abs_mean,c1_forgetting_mae_abs_std,c1_forgetting_mae_abs_count,c1_forgetting_mae_rel_%_mean,c1_forgetting_mae_rel_%_std,c1_forgetting_mae_rel_%_count,c2_mae_mean,c2_mae_std,c2_mae_count
0,fast,ewc,rate,1e+07,1.000000e+07,0.084224,0.004036,3,0.120972,0.060399,3,0.036748,0.063703,3,45.730858,79.269036,3,0.179906,0.002547,3
1,fast,ewc,rate,3e+07,3.000000e+07,0.084224,0.004036,3,0.087397,0.003124,3,0.003173,0.005496,3,3.948347,6.839094,3,0.177774,0.006029,3
2,fast,ewc,rate,1e+08,1.000000e+08,0.086497,0.001687,3,0.093487,0.009361,3,0.006990,0.010988,3,8.244690,12.989080,3,0.179121,0.003081,3
3,fast,ewc,rate,3e+08,3.000000e+08,0.078499,0.007403,3,0.092174,0.007316,3,0.013675,0.011541,3,18.216163,15.261237,3,0.182258,0.013450,3
4,fast,ewc,rate,1e+09,1.000000e+09,0.078499,0.007403,3,0.083076,0.007625,3,0.004577,0.010584,3,6.481414,14.679183,3,0.200835,0.042485,3
5,fast,ewc,rate,1e+10,1.000000e+10,0.078499,0.007403,3,0.084110,0.010281,3,0.005612,0.014415,3,8.074686,19.931359,3,0.199521,0.033226,3
6,fast,ewc,rate,1e+11,1.000000e+11,0.084224,0.004036,3,0.084219,0.004047,3,-0.000005,0.000014,3,-0.006691,0.015948,3,0.179795,0.002647,3
7,fast,naive,rate,NaN,NaN,0.078499,0.007403,3,0.092098,0.014063,3,0.013599,0.020945,3,18.988183,27.316093,3,0.187488,0.011750,3
8,std,ewc,rate,1e+07,1.000000e+07,0.079383,0.002520,3,0.206634,0.119772,3,0.127252,0.122031,3,163.314495,158.761769,3,0.165350,0.002414,3
9,std,ewc,rate,3e+07,3.000000e+07,0.079383,0.002520,3,0.147377,0.048773,3,0.067994,0.050970,3,86.897645,66.889941,3,0.168813,0.004927,3


In [13]:
# ====================== Formateo para la memoria (tabla compacta) ======================

def fmt(x, prec=4):
    # Redondea y gestiona NaN de forma amigable
    import pandas as pd
    return "" if pd.isna(x) else f"{x:.{prec}f}"

show = agg.copy()

# 1) Crea 'count' a partir de cualquiera de las columnas *_count
count_cols = [c for c in show.columns if c.endswith("_count")]
if count_cols:
    show["count"] = show[count_cols[0]].astype("Int64")  # todas deberían coincidir
    # (opcional) elimina las columnas *_count individuales
    show = show.drop(columns=count_cols)

# 2) Redondea columnas de medias/desviaciones
for c in [c for c in show.columns if c.endswith("_mean") or c.endswith("_std")]:
    show[c] = show[c].map(lambda v: fmt(v, 4))

# 3) Selección de columnas clave (ajusta el orden a tu gusto)
cols = [
    "preset", "method", "encoder", "lambda",
    "c1_mae_mean", "c1_forgetting_mae_rel_%_mean", "c2_mae_mean",
    "c1_mae_std",  "c1_forgetting_mae_rel_%_std",  "c2_mae_std",
    "count"
]

# Si alguna columna no existiera (según tus métricas), la ignoramos con aviso
missing = [c for c in cols if c not in show.columns]
if missing:
    print("Aviso: faltan columnas en 'show':", missing)
    cols = [c for c in cols if c in show.columns]

show = show[cols].rename(columns={
    "preset": "preset",
    "method": "método",
    "encoder": "codificador",
    "lambda": "λ",
    "c1_mae_mean": "MAE Tarea1 (media)",
    "c1_forgetting_mae_rel_%_mean": "Olvido T1 (%) (media)",
    "c2_mae_mean": "MAE Tarea2 (media)",
    "c1_mae_std": "MAE Tarea1 (σ)",
    "c1_forgetting_mae_rel_%_std": "Olvido T1 (%) (σ)",
    "c2_mae_std": "MAE Tarea2 (σ)",
    "count": "n (semillas)"
})

show


,preset,método,codificador,λ,MAE Tarea1 (media),Olvido T1 (%) (media),MAE Tarea2 (media),MAE Tarea1 (σ),Olvido T1 (%) (σ),MAE Tarea2 (σ),n (semillas)
0,fast,ewc,rate,1e+07,0.0842,45.7309,0.1799,0.0040,79.2690,0.0025,3
1,fast,ewc,rate,3e+07,0.0842,3.9483,0.1778,0.0040,6.8391,0.0060,3
2,fast,ewc,rate,1e+08,0.0865,8.2447,0.1791,0.0017,12.9891,0.0031,3
3,fast,ewc,rate,3e+08,0.0785,18.2162,0.1823,0.0074,15.2612,0.0135,3
4,fast,ewc,rate,1e+09,0.0785,6.4814,0.2008,0.0074,14.6792,0.0425,3
5,fast,ewc,rate,1e+10,0.0785,8.0747,0.1995,0.0074,19.9314,0.0332,3
6,fast,ewc,rate,1e+11,0.0842,-0.0067,0.1798,0.0040,0.0159,0.0026,3
7,fast,naive,rate,NaN,0.0785,18.9882,0.1875,0.0074,27.3161,0.0118,3
8,std,ewc,rate,1e+07,0.0794,163.3145,0.1653,0.0025,158.7618,0.0024,3
9,std,ewc,rate,3e+07,0.0794,86.8976,0.1688,0.0025,66.8899,0.0049,3
